# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 3 2022 9:43AM,238732,20,R1-8100042,Reliable 1 Laboratories LLC,Released
1,Jun 3 2022 9:41AM,238731,20,ALU0006972,Alumier Labs Inc.,Released
2,Jun 3 2022 9:41AM,238731,20,ALU0006973,Alumier Labs Inc.,Released
3,Jun 3 2022 9:41AM,238731,20,ALUR165535107,Alumier Labs Inc.,Released
4,Jun 3 2022 9:41AM,238731,20,ALUR229923628,Alumier Labs Inc.,Released
5,Jun 3 2022 9:41AM,238731,20,ALUR768720644,Alumier Labs Inc.,Released
6,Jun 3 2022 9:41AM,238731,20,ALUR805797152,Alumier Labs Inc.,Released
7,Jun 3 2022 9:41AM,238731,20,ALUR864188519,Alumier Labs Inc.,Released
8,Jun 3 2022 9:41AM,238731,20,ALUR236351441,Alumier Labs Inc.,Released
9,Jun 3 2022 9:41AM,238731,20,ALUR597681417,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,238727,Released,32
32,238729,Released,9
33,238730,Released,1
34,238731,Released,18
35,238732,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238727,NaN,NaN,32.0
238729,NaN,NaN,9.0
238730,NaN,NaN,1.0
238731,NaN,NaN,18.0
238732,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238634,0.0,1.0,0.0
238643,0.0,2.0,15.0
238644,0.0,1.0,0.0
238647,0.0,0.0,1.0
238649,15.0,8.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238634,0,1,0
238643,0,2,15
238644,0,1,0
238647,0,0,1
238649,15,8,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238634,0,1,0
1,238643,0,2,15
2,238644,0,1,0
3,238647,0,0,1
4,238649,15,8,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238634,,1,
1,238643,,2,15
2,238644,,1,
3,238647,,,1
4,238649,15,8,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 3 2022 9:43AM,238732,20,Reliable 1 Laboratories LLC
1,Jun 3 2022 9:41AM,238731,20,Alumier Labs Inc.
19,Jun 3 2022 9:23AM,238730,12,Hush Hush
20,Jun 3 2022 9:23AM,238729,12,Hush Hush
29,Jun 3 2022 9:10AM,238727,10,ISDIN Corporation
61,Jun 3 2022 8:56AM,238724,10,ISDIN Corporation
115,Jun 3 2022 8:50AM,238723,10,ISDIN Corporation
158,Jun 3 2022 8:39AM,238722,10,ISDIN Corporation
179,Jun 3 2022 7:58AM,238718,19,"Graystone, LLC"
181,Jun 3 2022 12:00AM,238715,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 3 2022 9:43AM,238732,20,Reliable 1 Laboratories LLC,,,1
1,Jun 3 2022 9:41AM,238731,20,Alumier Labs Inc.,,,18
2,Jun 3 2022 9:23AM,238730,12,Hush Hush,,,1
3,Jun 3 2022 9:23AM,238729,12,Hush Hush,,,9
4,Jun 3 2022 9:10AM,238727,10,ISDIN Corporation,,,32
5,Jun 3 2022 8:56AM,238724,10,ISDIN Corporation,,,54
6,Jun 3 2022 8:50AM,238723,10,ISDIN Corporation,,,43
7,Jun 3 2022 8:39AM,238722,10,ISDIN Corporation,,5,16
8,Jun 3 2022 7:58AM,238718,19,"Graystone, LLC",,,2
9,Jun 3 2022 12:00AM,238715,10,Eywa Pharma Inc.,2,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 9:43AM,238732,20,Reliable 1 Laboratories LLC,1,,
1,Jun 3 2022 9:41AM,238731,20,Alumier Labs Inc.,18,,
2,Jun 3 2022 9:23AM,238730,12,Hush Hush,1,,
3,Jun 3 2022 9:23AM,238729,12,Hush Hush,9,,
4,Jun 3 2022 9:10AM,238727,10,ISDIN Corporation,32,,
5,Jun 3 2022 8:56AM,238724,10,ISDIN Corporation,54,,
6,Jun 3 2022 8:50AM,238723,10,ISDIN Corporation,43,,
7,Jun 3 2022 8:39AM,238722,10,ISDIN Corporation,16,5,
8,Jun 3 2022 7:58AM,238718,19,"Graystone, LLC",2,,
9,Jun 3 2022 12:00AM,238715,10,Eywa Pharma Inc.,1,,2


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 9:43AM,238732,20,Reliable 1 Laboratories LLC,1,,
1,Jun 3 2022 9:41AM,238731,20,Alumier Labs Inc.,18,,
2,Jun 3 2022 9:23AM,238730,12,Hush Hush,1,,
3,Jun 3 2022 9:23AM,238729,12,Hush Hush,9,,
4,Jun 3 2022 9:10AM,238727,10,ISDIN Corporation,32,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 9:43AM,238732,20,Reliable 1 Laboratories LLC,1.0,NaN,NaN
1,Jun 3 2022 9:41AM,238731,20,Alumier Labs Inc.,18.0,NaN,NaN
2,Jun 3 2022 9:23AM,238730,12,Hush Hush,1.0,NaN,NaN
3,Jun 3 2022 9:23AM,238729,12,Hush Hush,9.0,NaN,NaN
4,Jun 3 2022 9:10AM,238727,10,ISDIN Corporation,32.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 9:43AM,238732,20,Reliable 1 Laboratories LLC,1.0,0.0,0.0
1,Jun 3 2022 9:41AM,238731,20,Alumier Labs Inc.,18.0,0.0,0.0
2,Jun 3 2022 9:23AM,238730,12,Hush Hush,1.0,0.0,0.0
3,Jun 3 2022 9:23AM,238729,12,Hush Hush,9.0,0.0,0.0
4,Jun 3 2022 9:10AM,238727,10,ISDIN Corporation,32.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3819118,197.0,17.0,21.0
12,954810,26.0,1.0,0.0
16,477298,1.0,1.0,0.0
19,477368,3.0,0.0,0.0
20,716145,24.0,15.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3819118,197.0,17.0,21.0
1,12,954810,26.0,1.0,0.0
2,16,477298,1.0,1.0,0.0
3,19,477368,3.0,0.0,0.0
4,20,716145,24.0,15.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,197.0,17.0,21.0
1,12,26.0,1.0,0.0
2,16,1.0,1.0,0.0
3,19,3.0,0.0,0.0
4,20,24.0,15.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,197.0
1,12,Released,26.0
2,16,Released,1.0
3,19,Released,3.0
4,20,Released,24.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20
Status,,,,,
Completed,21.0,0.0,0.0,0.0,0.0
Executing,17.0,1.0,1.0,0.0,15.0
Released,197.0,26.0,1.0,3.0,24.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20
0,Completed,21.0,0.0,0.0,0.0,0.0
1,Executing,17.0,1.0,1.0,0.0,15.0
2,Released,197.0,26.0,1.0,3.0,24.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20
0,Completed,21.0,0.0,0.0,0.0,0.0
1,Executing,17.0,1.0,1.0,0.0,15.0
2,Released,197.0,26.0,1.0,3.0,24.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()